In [ ]:
import minimalmodbus
import serial
import time

# ——————————————————————————————————————
# 1. ตั้งค่าพอร์ต Serial
# ——————————————————————————————————————
PORT    = '/dev/ttyUSB0'    # หรือ 'COM4' บน Windows
BAUD    = 19200
TIMEOUT = 1                 # วินาที

# ——————————————————————————————————————
# 2. กำหนดเซ็นเซอร์แต่ละตัว (Slave ID ตามภาพ)
# ——————————————————————————————————————
SENSORS = [
    {'id': 1,  'name': 'Hydro-Mix #1'},
    
]

# ——————————————————————————————————————
# 3. กำหนดรายการ Register ที่ต้องอ่าน
#    one_based: Address แบบ one-based
#    decimals : จำนวนทศนิยมที่ต้องการ (0 = เลยค่า raw ดิบ)
#    fc       : Function code (4 = Input Registers)
#    label    : ชื่อที่จะพิมพ์
#    unit     : หน่วย (เช่น '%', '°C' ฯลฯ)
# ——————————————————————————————————————
REGISTERS = [
    {'one_based': 30014, 'decimals': 0, 'fc': 4, 'label': 'Raw Moisture','unit': ''},
]

# ——————————————————————————————————————
# 4. สร้าง Instrument object สำหรับแต่ละเซ็นเซอร์
# ——————————————————————————————————————
instruments = {}
for s in SENSORS:
    inst = minimalmodbus.Instrument(PORT, s['id'], mode=minimalmodbus.MODE_RTU)
    inst.serial.baudrate = BAUD
    inst.serial.bytesize   = 8
    inst.serial.parity     = serial.PARITY_NONE
    inst.serial.stopbits   = 1
    inst.serial.timeout    = TIMEOUT
    inst.clear_buffers_before_each_transaction = True
    # เปิด debug ดู telegram request/response (ถ้าต้องการ)
    # inst.debug = True
    instruments[s['id']] = inst

# ——————————————————————————————————————
# 5. ฟังก์ชันช่วยอ่าน Register และแปลงเป็น zero-based address
# ——————————————————————————————————————
def read_one(inst, one_based, decimals, fc):
    # สมมติว่าเป็น Input Registers (3000x หรือ 5000x ก็ลบด้วยค่า one-based offset เดียวกัน)
    zero_addr = one_based - one_based
    return inst.read_register(zero_addr, decimals, functioncode=fc)

# ——————————————————————————————————————
# 6. ลูปอ่านค่าต่อเนื่อง
# ——————————————————————————————————————
try:
    while True:
        print(time.strftime('%Y-%m-%d %H:%M:%S'))
        for s in SENSORS:
            inst = instruments[s['id']]
            print(f"--- {s['name']} (ID={s['id']}) ---")
            for reg in REGISTERS:
                try:
                    val = read_one(inst,
                                   reg['one_based'],
                                   reg['decimals'],
                                   reg['fc'])
                    # พิมพ์ค่าพร้อมทศนิยมและหน่วย
                    fmt = f"{{:.{reg['decimals']}f}}"
                    print(f"{reg['label']:25}: {fmt.format(val)} {reg['unit']}")
                except minimalmodbus.NoResponseError:
                    print(f"{reg['label']:25}: ERROR – no response")
                except Exception as e:
                    print(f"{reg['label']:25}: ERROR – {e}")
        print('-' * 40)
        time.sleep(1)  # อ่านซ้ำทุก 1 วินาที
except KeyboardInterrupt:
    print("Stopped by user")

2025-05-19 17:06:28
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1726 
----------------------------------------
2025-05-19 17:06:30
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1729 
----------------------------------------
2025-05-19 17:06:31
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1736 
----------------------------------------
2025-05-19 17:06:32
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1750 
----------------------------------------
2025-05-19 17:06:33
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1724 
----------------------------------------
2025-05-19 17:06:34
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1748 
----------------------------------------
2025-05-19 17:06:35
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : 1744 
----------------------------------------
2025-05-19 17:06:36
--- Hydro-Mix #1 (ID=1) ---
Raw Moisture             : ERROR – Checksum error in rtu mode: b'\xb0\xbb' instead of b'\xd8S' . T

In [2]:
# /app/services/sensor_service.py
import logging
import asyncio
from app.services.sensor_control import MoistureSensor

logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')

sensor = MoistureSensor()
current_key = 2001

async def continuous_data_reading(queue: str) -> dict:
    """
    อ่านค่า raw sensor ต่อเนื่องจนกว่าจะเจอค่า <=10 แล้วคืน dict ของค่าที่เก็บไว้
    """
    global current_key
    sensor_data = {}
    logging.info(f"Starting raw data collection for queue: {queue}")

    while True:
        result = sensor.read_data(current_key)
        if result:
            key, value = result
            if value > 10:
                sensor_data[str(key)] = value
                logging.info(f"Read raw value: {value} at key: {key}")
                current_key += 1
            elif sensor_data:
                logging.info("Value <= 10 and data collected, stopping.")
                break
        else:
            logging.warning("No sensor data available at the moment.")

        await asyncio.sleep(1)

    logging.info(f"Completed raw data: {sensor_data}")
    return sensor_data

def get_sensor_status():
    """
    คืนสถานะล่าสุด (key, value) หรือ {'status':'unavailable'}
    """
    result = sensor.read_data(current_key)
    if result:
        key, value = result
        return {"key": key, "value": value}
    return {"status": "unavailable"}
continuous_data_reading()

2025-05-19 17:15:11,677 INFO MoistureSensor initialized on /dev/ttyUSB0 (ID=1), one-based register=30013, zero-address=12


TypeError: continuous_data_reading() missing 1 required positional argument: 'queue'

In [10]:
import os
from dotenv import load_dotenv
import pandas as pd
from app.services.sensor_control import MoistureSensor

# Load your sensor environment
load_dotenv('.env.sensor')  # adjust path as needed

# Instantiate and read multiple samples
sensor = MoistureSensor()
results = []
for key_offset in range(1, 10000):
    res = sensor.read_data(start_key=key_offset)
    if res:
        key, val = res
        results.append({'key': key, 'moisture': val})
    else:
        results.append({'key': key_offset, 'moisture': None})
sensor.close_connection()

# Display results
df = pd.DataFrame(results)
df


2025-05-16 17:13:31,979 INFO MoistureSensor initialized on /dev/ttyUSB0 (ID=1), register=30013
2025-05-16 17:13:45,454 WARNING MoistureSensor: failed to read register → Checksum error in rtu mode: b'\x00\xb9' instead of b'XQ' . The response is: b'\x01\x04\x04\x02\x00\x00\xb9' (plain response: b'\x01\x04\x04\x02\x00\x00\xb9')


KeyboardInterrupt: 